In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler,LabelEncoder

def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocesses the data for companies.

    Args:
        companies: Raw data.
    Returns:
        Preprocessed data, with `company_rating` converted to a float and
        `iata_approved` converted to boolean.
    """
    diabetes = datasets.load_diabetes()
    df = pd.DataFrame(diabetes.data, columns=("age", "sex", "bmi", "map", "tc", "ldl", "hdl", "tch", "ltg", "glu"))
    df['target'] = diabetes.target
    df['target_class']=0
    df['target_class']=np.where(df['target']>211.5,1,0)

    features=[c for c in df.columns if c not in ['target','target_class']]
    return df

{25.0, 31.0, 37.0, 39.0, 40.0, 42.0, 43.0, 44.0, 45.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 57.0, 58.0, 59.0, 60.0, 61.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 77.0, 78.0, 79.0, 80.0, 81.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 131.0, 132.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 158.0, 160.0, 161.0, 162.0, 163.0, 164.0, 166.0, 167.0, 168.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0, 177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 183.0, 184.0, 185.0, 186.0, 187.0, 189.0, 190.0, 191.0, 192.0, 195.0, 196.0, 197.0, 198.0, 199.0, 200.0, 201.0, 202.0, 206.0, 208.0, 209.0, 210.0, 212.0, 214.0, 

In [2]:
df

,age,sex,bmi,map,tc,ldl,hdl,tch,ltg,glu,target,target_class
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0,0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0,0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0,0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0,0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0,0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0,0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0,0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0,1


In [3]:
# !pip install catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from catboost import Pool

In [4]:
importance_df=pd.DataFrame(index=df[features].columns)
fold=3
sfk=StratifiedKFold(n_splits=fold, shuffle=True, random_state=2)
for i ,(tr_index,va_index) in enumerate(sfk.split(df,df['target_class'])):
  tr_x,tr_y=df[features].iloc[tr_index],df['target_class'].iloc[tr_index]
  va_x,va_y=df[features].iloc[va_index],df['target_class'].iloc[va_index]
  tr_set=Pool(tr_x,label=tr_y)
  va_set=Pool(va_x,label=va_y)
  model=CatBoostClassifier(iterations=100000,random_state=2,nan_mode='Min',eval_metric='AUC')#,task_type='GPU')
  # ↑実験管理：パラメータの管理を行う
  model.fit(tr_set,eval_set=[va_set],verbose_eval =1000,use_best_model=True,early_stopping_rounds=500)
# ↑で実際にモデルを作って重要度の結果を算出している
  importance_df['importance']=model.feature_importances_/fold
importance_df.sort_values('importance',inplace=True,ascending=False)
importance_df.reset_index(inplace=True)
importance_df



Learning rate set to 0.003159
0:	test: 0.8363769	best: 0.8363769 (0)	total: 113ms	remaining: 3h 8m 59s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.856586316
bestIteration = 9

Shrink model to first 10 iterations.
Learning rate set to 0.003162
0:	test: 0.7993857	best: 0.7993857 (0)	total: 3.29ms	remaining: 5m 29s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8906633907
bestIteration = 5

Shrink model to first 6 iterations.
Learning rate set to 0.003162
0:	test: 0.8589681	best: 0.8589681 (0)	total: 2.92ms	remaining: 4m 51s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9117936118
bestIteration = 126

Shrink model to first 127 iterations.


,index,importance
0,bmi,7.498721
1,ltg,7.134541
2,map,6.622075
3,tch,2.332155
4,ldl,1.853972
5,sex,1.807728
6,tc,1.733162
7,glu,1.638408
8,hdl,1.544259
9,age,1.168311


In [5]:
importance_df=pd.DataFrame(index=df[features].columns)
fold=3
sfk=StratifiedKFold(n_splits=fold, shuffle=True, random_state=2)
for i ,(tr_index,va_index) in enumerate(sfk.split(df,df['target'])):
  tr_x,tr_y=df[features].iloc[tr_index],df['target'].iloc[tr_index]
  va_x,va_y=df[features].iloc[va_index],df['target'].iloc[va_index]

  model=CatBoostRegressor(iterations=100000,random_state=2,nan_mode='Min',eval_metric='RMSE')#,task_type='GPU')
  model.fit(tr_x,tr_y,eval_set=[(va_x,va_y)],verbose_eval =1000,use_best_model=True,early_stopping_rounds=500)
  # ↑で実際にモデルを作って重要度の結果を算出している
  
  importance_df['importance']=model.feature_importances_/fold
importance_df.sort_values('importance',inplace=True,ascending=False)
importance_df.reset_index(inplace=True)
importance_df

Learning rate set to 0.002532
0:	learn: 76.0922119	test: 78.5717922	best: 78.5717922 (0)	total: 1.78ms	remaining: 2m 58s


c:\Users\ymiyawaki\AppData\Local\miniconda3\envs\pythonenv2\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


1000:	learn: 45.7568356	test: 58.1113611	best: 58.1113611 (1000)	total: 1.51s	remaining: 2m 29s
2000:	learn: 37.4537057	test: 56.8579856	best: 56.8470643 (1910)	total: 3.26s	remaining: 2m 39s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 56.80764302
bestIteration = 2399

Shrink model to first 2400 iterations.
Learning rate set to 0.002533
0:	learn: 77.3125292	test: 76.1748500	best: 76.1748500 (0)	total: 2.19ms	remaining: 3m 38s
1000:	learn: 45.5589903	test: 57.8126317	best: 57.8126317 (1000)	total: 1.69s	remaining: 2m 47s
2000:	learn: 37.6175987	test: 57.0279367	best: 57.0209867 (1977)	total: 3.25s	remaining: 2m 39s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 57.01448393
bestIteration = 2140

Shrink model to first 2141 iterations.
Learning rate set to 0.002533
0:	learn: 77.3664351	test: 76.0452779	best: 76.0452779 (0)	total: 1.97ms	remaining: 3m 16s
1000:	learn: 46.9345203	test: 55.4303858	best: 55.4303858 (1000)	total: 1.39s	remaining: 2m 17

,index,importance
0,ltg,7.823661
1,bmi,7.210640
2,map,4.896306
3,age,2.423447
4,glu,2.392972
5,hdl,1.994042
6,sex,1.831077
7,tch,1.821389
8,ldl,1.717503
9,tc,1.222297
